In [1]:
import os
os.environ["PYTHONHASHSEED"] = "0"
os.environ["TF_DETERMINISTIC_OPS"] = "1"     # deterministic GPU ops
os.environ["TF_CUDNN_DETERMINISTIC"] = "1"   # deterministic cuDNN kernels

import tensorflow as tf
from tensorflow import keras
from keras import layers
from pathlib import Path
import numpy as np

# Set the seeds for reproducibility
import random
SEED = 2648509283
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)
tf.keras.utils.set_random_seed(SEED)
tf.config.experimental.enable_op_determinism()

/Users/eugeneshevchenko/Courses/RD_Computer-Vision/HW/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [4]:
DATASET_DIR = "../data/chest_xray"
DATASET_PATH = Path(DATASET_DIR)
TRAIN_PATH = DATASET_PATH / "train"
TEST_PATH = DATASET_PATH / "test"
CLASSES = ["NORMAL", "PNEUMONIA"]
ID_TO_CLASS = {0:'NORMAL', 1:'PNEUMONIA'}
IMG_SIZE_SIDE = 80
IMG_SIZE = (IMG_SIZE_SIDE, IMG_SIZE_SIDE)
BATCH = 32

In [12]:
import optuna
from sklearn.model_selection import StratifiedKFold
from collections import Counter

AUTO = tf.data.AUTOTUNE
INPUT_SHAPE = IMG_SIZE + (1,)
FOLDS = 5
EPOCHS = 100
monitor_metric = "val_AUC_PR"


def list_dir_images(root_dir, classes):
    root = Path(root_dir)
    filepaths, labels = [], []
    for li, cls in enumerate(classes):
        for p in sorted((root / cls).rglob("*")):
            if p.suffix.lower() in {".jpg", ".jpeg"}:
                filepaths.append(str(p))
                labels.append(li)  # 0..C-1
    return np.array(filepaths), np.array(labels, dtype=np.int64)

filepaths, labels = list_dir_images(TRAIN_PATH, CLASSES)


### Preprocessing to match your loader
def decode_grayscale(path):
    img = tf.io.read_file(path)
    img = tf.image.decode_image(img, channels=1, expand_animations=False)
    img = tf.image.convert_image_dtype(img, tf.float32)
    img = tf.image.resize(img, IMG_SIZE, method="bilinear")
    return img


def make_norm_layer(train_paths):
    norm = layers.Normalization(axis=-1)
    ds = (
        tf.data.Dataset.from_tensor_slices(train_paths)
          .map(lambda p: decode_grayscale(p), num_parallel_calls=AUTO)
          .batch(256).prefetch(AUTO)
    )
    norm.adapt(ds)
    return norm


def make_ds(paths, ys, batch_size=BATCH, training=True, norm_layer=None, aug=True,
            rot=0.04, zoom=0.05, contrast=0.05, brightness=0.05):
    ds = tf.data.Dataset.from_tensor_slices((paths, ys))

    # Augmentation
    aug_layers = keras.Sequential([
        layers.RandomRotation(rot, fill_mode="constant", seed=SEED),
        layers.RandomZoom(zoom, fill_mode="constant", seed=SEED),
        layers.RandomContrast(contrast, seed=SEED),
        layers.RandomBrightness(brightness, seed=SEED),
    ]) if aug else None

    def _map(path, y):
        x = decode_grayscale(path)
        if training and aug_layers is not None:
            x = aug_layers(x)
        # if norm_layer is not None:
        #     x = norm_layer(x)

        y = tf.cast(y, tf.float32)
        y = tf.expand_dims(y, -1)

        return x, y

    if training:
        ds = ds.shuffle(min(len(paths), 10000), seed=SEED, reshuffle_each_iteration=True)
    ds = ds.map(_map, num_parallel_calls=AUTO).batch(batch_size).prefetch(AUTO)
    return ds


def compute_class_weights(y):
    c = Counter(y.tolist())
    total = len(y)
    w0 = total / (2.0 * c.get(0, 1))
    w1 = total / (2.0 * c.get(1, 1))
    return {0: w0, 1: w1}


def build_cnn(input_shape=INPUT_SHAPE, kernel_size_1=3, dropout_rate=0.3, l2=0.0):
    reg = keras.regularizers.l2(l2) if l2 and l2 > 0 else None

    inputs = keras.Input(shape=input_shape)

    x = layers.Conv2D(32, kernel_size_1, strides=2, padding='same', use_bias=False, kernel_regularizer=reg)(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    x = layers.Conv2D(64, 3, strides=2, padding='same', use_bias=False, kernel_regularizer=reg)(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    x = layers.Conv2D(128, 3, strides=2, padding='same', use_bias=False, kernel_regularizer=reg)(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    x = layers.Conv2D(128, 3, padding='same', use_bias=False, kernel_regularizer=reg)(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(dropout_rate, seed=SEED)(x)
    x = layers.Dense(1, activation="sigmoid")(x)

    return keras.Model(inputs, x)


### Optuna objective with K-fold CV
def objective(trial: optuna.Trial):
    cw_mode = trial.suggest_categorical("class_weight_mode", ["none", "balanced", "custom-1", "custom-2"])
    lr = trial.suggest_float("lr", 1e-5, 3e-3, log=True)

    batch = trial.suggest_categorical("batch", [16, 32, 64, 96])
    kernel_size_1 = trial.suggest_categorical("kernel_size_1", [3, 5])
    drop = trial.suggest_float("dropout", 0.2, 0.5)
    l2 = trial.suggest_categorical("l2", [0.0, 1e-6, 3e-6, 1e-5, 3e-5, 1e-4, 3e-4, 1e-3])
    rot = trial.suggest_float("rot", 0.0, 0.08)
    zoom = trial.suggest_float("zoom", 0.0, 0.10)
    contrast = trial.suggest_float("contrast", 0.0, 0.15)
    brightness = trial.suggest_float("brightness", 0.0, 0.10)

    skf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=SEED)
    fold_scores = []

    for fold, (tr_idx, va_idx) in enumerate(skf.split(filepaths, labels), start=1):
        tr_paths = filepaths[tr_idx]
        va_paths = filepaths[va_idx]
        y_tr = labels[tr_idx]
        y_va = labels[va_idx]

        norm = make_norm_layer(tr_paths)

        ds_tr = make_ds(tr_paths, y_tr, batch_size=batch, training=True,  norm_layer=norm,
                        aug=True, rot=rot, zoom=zoom, contrast=contrast, brightness=brightness)
        ds_va = make_ds(va_paths, y_va, batch_size=batch, training=False, norm_layer=norm, aug=False)

        model = build_cnn(kernel_size_1=kernel_size_1, dropout_rate=drop, l2=l2)
        model.compile(
            optimizer=keras.optimizers.Adam(lr),
            loss=keras.losses.BinaryCrossentropy(),
            metrics=[
                keras.metrics.AUC(curve='PR', name='AUC_PR'),
                keras.metrics.AUC(curve='ROC', name='AUC_ROC'),
                keras.metrics.Recall(name='recall'),
                keras.metrics.Precision(name='precision'),
                keras.metrics.F1Score(name='F1'),
                keras.metrics.FBetaScore(name='F2', beta=2.0),
            ],
        )

        early = keras.callbacks.EarlyStopping(monitor=monitor_metric, mode="max",
                                              patience=6, restore_best_weights=True)
        rlr = keras.callbacks.ReduceLROnPlateau(monitor=monitor_metric, mode="max",
                                                factor=0.3, patience=2, min_lr=1e-6)

        if cw_mode == "none":
            cw = None
        elif cw_mode == "balanced":
            cw = compute_class_weights(y_tr)
        elif cw_mode == "custom-1":
            cw = {0: 1.0, 1: 1.25}
        else:
            cw = {0: 1.5, 1: 1.0}


        hist = model.fit(
            ds_tr,
            validation_data=ds_va,
            epochs=EPOCHS,
            class_weight=cw,
            callbacks=[early, rlr],
            verbose=0,
        )

        best_v = float(np.max(hist.history[monitor_metric]))
        best_fold_metrics = {
            "AUC_PR": best_v,
            "loss": float(np.min(hist.history["val_loss"])),
            "recall": float(np.max(hist.history["val_recall"])),
            "precision": float(np.max(hist.history["val_precision"])),
            "F1": float(np.max(hist.history["val_F1"])),
        }
        fold_scores.append(best_fold_metrics)

        keras.backend.clear_session()

    mean_auc = np.mean([f["AUC_PR"] for f in fold_scores])
    mean_loss = np.mean([f["loss"] for f in fold_scores])
    mean_recall = np.mean([f["recall"] for f in fold_scores])
    mean_prec = np.mean([f["precision"] for f in fold_scores])
    mean_f1 = np.mean([f["F1"] for f in fold_scores])

    trial.set_user_attr("cv_std", float(np.std([f["AUC_PR"] for f in fold_scores])))
    trial.set_user_attr("mean_loss", mean_loss)
    trial.set_user_attr("mean_recall", mean_recall)
    trial.set_user_attr("mean_precision", mean_prec)
    trial.set_user_attr("mean_f1", mean_f1)

    return mean_auc

### Run Optuna
study = optuna.create_study(direction="maximize",
                            sampler=optuna.samplers.TPESampler(seed=SEED),
                            pruner=optuna.pruners.MedianPruner(n_startup_trials=5, n_warmup_steps=5))
study.optimize(objective, n_trials=40, show_progress_bar=True)

print("Best CV mean AUC_PR:", study.best_value)
print("Best params:", study.best_params)

best = study.best_trial
print("Best CV mean AUC_PR:", best.value)
print("Best CV std AUC_PR:", best.user_attrs.get("cv_std"))
print("Best mean loss:", best.user_attrs.get("mean_loss"))
print("Best mean Recall:", best.user_attrs.get("mean_recall"))
print("Best mean Precision:", best.user_attrs.get("mean_precision"))
print("Best mean F1:", best.user_attrs.get("mean_f1"))
print("Best params:", best.params)

"""
Check STD:
- Low std (≈0.01–0.02) - stable model, good generalization across folds
- High std (≥0.05–0.10) - possibly overfits or data imbalance issues
"""

### Train final model on all data with best params
bp = study.best_params
norm_all = make_norm_layer(filepaths)

best_cw_mode = bp.get("class_weight_mode", "none")

if best_cw_mode == "none":
    final_cw = None
elif best_cw_mode == "balanced":
    final_cw = compute_class_weights(labels)
elif best_cw_mode == "custom-1":
    final_cw = {0: 1.0, 1: 1.25}
else:
    final_cw = {0: 1.5, 1: 1.0}

ds_all = make_ds(filepaths, labels,
                 batch_size=bp.get("batch", BATCH),
                 training=True, norm_layer=norm_all, aug=True,
                 rot=bp.get("rot", 0.04),
                 zoom=bp.get("zoom", 0.05),
                 contrast=bp.get("contrast", 0.05),
                 brightness=bp.get("brightness", 0.05))

final_model = build_cnn(kernel_size_1=bp.get("kernel_size_1", 5), dropout_rate=bp.get("dropout", 0.3), l2=bp.get("l2", 0.0))
final_model.compile(optimizer=keras.optimizers.Adam(bp.get("lr", 3e-4)),
                    loss=keras.losses.BinaryCrossentropy(),
                    metrics=[keras.metrics.AUC(curve='PR', name='AUC_PR'),
                             keras.metrics.AUC(curve='ROC', name='AUC_ROC'),
                             keras.metrics.Recall(name='recall'),
                             keras.metrics.Precision(name='precision'),
                             keras.metrics.F1Score(name='F1'),
                             keras.metrics.FBetaScore(name='F2', beta=2.0)])
ckpt = keras.callbacks.ModelCheckpoint("cv-final-best.keras", monitor="AUC_PR", mode="max", save_best_only=True)
final_model.fit(ds_all, class_weight=final_cw, epochs=EPOCHS, callbacks=[ckpt], verbose=1)

[I 2025-11-12 19:32:03,531] A new study created in memory with name: no-name-ee38f6d6-ed67-414a-a3d4-f5298be10060
  0%|          | 0/40 [00:00<?, ?it/s]2025-11-12 19:32:04.140685: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-11-12 19:35:13.978040: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-11-12 19:36:55.355570: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-11-12 19:40:26.004104: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-11-12 19:41:28.963605: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
Best trial: 0. Best value: 0.880308:   2%|▎         | 1/40 [11:17<7:20:10, 677.19s/

[I 2025-11-12 19:43:20,724] Trial 0 finished with value: 0.8803077697753906 and parameters: {'class_weight_mode': 'none', 'lr': 0.00015488826038852235, 'batch': 32, 'kernel_size_1': 5, 'dropout': 0.22141577305870708, 'l2': 3e-06, 'rot': 0.06719957694928133, 'zoom': 0.08750126244896306, 'contrast': 0.10430744986199167, 'brightness': 0.08048906763427496}. Best is trial 0 with value: 0.8803077697753906.


2025-11-12 19:45:12.942752: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-11-12 19:47:15.761106: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-11-12 19:50:00.964652: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-11-12 19:53:51.296660: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
Best trial: 1. Best value: 0.965131:   5%|▌         | 2/40 [23:58<7:40:03, 726.42s/it]2025-11-12 19:56:01.793254: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


[I 2025-11-12 19:56:01,604] Trial 1 finished with value: 0.9651314854621887 and parameters: {'class_weight_mode': 'custom-1', 'lr': 3.176058944564007e-05, 'batch': 96, 'kernel_size_1': 5, 'dropout': 0.4004113923236142, 'l2': 3e-06, 'rot': 0.003553326499556411, 'zoom': 0.05925087757474243, 'contrast': 0.01522666153724138, 'brightness': 0.04192847299270711}. Best is trial 1 with value: 0.9651314854621887.


2025-11-12 19:57:19.814780: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-11-12 19:59:46.231066: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-11-12 20:01:02.031157: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-11-12 20:02:19.350839: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
Best trial: 1. Best value: 0.965131:   8%|▊         | 3/40 [31:44<6:14:37, 607.50s/it]

...

2025-11-13 03:21:52.155366: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-11-13 03:25:12.606474: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-11-13 03:32:15.580962: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-11-13 03:39:52.168513: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-11-13 03:45:41.858653: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
Best trial: 14. Best value: 0.992779:  65%|██████▌   | 26/40 [8:20:18<6:55:19, 1779.93s/it]

[I 2025-11-13 03:52:22,094] Trial 25 finished with value: 0.9745326399803161 and parameters: {'class_weight_mode': 'none', 'lr': 4.2025761589450045e-05, 'batch': 96, 'kernel_size_1': 3, 'dropout': 0.28167030671666893, 'l2': 1e-06, 'rot': 0.020006223608002856, 'zoom': 0.07081473147890299, 'contrast': 0.062416009509071474, 'brightness': 0.019050672682768707}. Best is trial 14 with value: 0.9927791595458985.


2025-11-13 03:52:22.407822: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-11-13 04:13:08.298156: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-11-13 04:20:11.162443: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
Best trial: 14. Best value: 0.992779:  65%|██████▌   | 26/40 [8:50:23<4:45:35, 1224.00s/it]


[W 2025-11-13 04:22:27,402] Trial 26 failed with parameters: {'class_weight_mode': 'custom-2', 'lr': 1.6940839132233863e-05, 'batch': 32, 'kernel_size_1': 3, 'dropout': 0.32523031300648453, 'l2': 3e-05, 'rot': 0.011524840492183327, 'zoom': 0.024889496208025993, 'contrast': 0.1495618834532588, 'brightness': 0.0003796130529414083} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/Users/eugeneshevchenko/Courses/RD_Computer-Vision/HW/.venv/lib/python3.9/site-packages/optuna/study/_optimize.py", line 205, in _run_trial
    value_or_values = func(trial)
  File "/var/folders/0v/tpjvqznj08x5pxjvqbf_1rmw0000gp/T/ipykernel_1643/2874915099.py", line 165, in objective
    hist = model.fit(
  File "/Users/eugeneshevchenko/Courses/RD_Computer-Vision/HW/.venv/lib/python3.9/site-packages/keras/src/utils/traceback_utils.py", line 117, in error_handler
    return fn(*args, **kwargs)
  File "/Users/eugeneshevchenko/Courses/RD_Computer-Vision/HW/.venv/lib/pyt

KeyboardInterrupt: 